In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
import nibabel as nib

In [2]:
main_path = r'F:\Project_AD'
df_dir = pd.read_csv(os.path.join(main_path, 'Code', 'DTI', 'Subject_DKI.txt'), header=None, names=['Subject', 'Time'])

In [6]:
t_dim = df_dir.shape[0]

main_path = r'F:\Project_AD\Data_proc\TBSS_DKI\stats'
FA = nib.load(os.path.join(main_path, 'all_FA_skeletonised.nii.gz'))
MD = nib.load(os.path.join(main_path, 'all_MD_skeletonised.nii.gz'))

In [8]:
df_subj = pd.DataFrame()

for index, row in tqdm(df_dir.iterrows()):
    subj = row['Subject']
    time_s = row['Time']
    
    FA_data = FA.dataobj[...,index]
    FA_mask = np.where(FA_data >= 0.3, 1.0, 0.0)
    MD_data = MD.dataobj[...,index]
    MD_analysis = MD_data[FA_mask == 1]
    p5 = np.nanpercentile(MD_analysis, 5)
    p95 = np.nanpercentile(MD_analysis, 95)
    percentile_widths = p95 - p5
    
    subject_data = pd.concat([pd.DataFrame([p95]), pd.DataFrame([p5]), pd.DataFrame([p95-p5])], axis=1)
    subject_data.index=pd.MultiIndex.from_product([[subj],[time_s]])
    df_subj = pd.concat([df_subj, subject_data], ignore_index=False)

407it [3:15:12, 28.78s/it]


In [9]:
df_subj.index.names=['PTID','Time']
df_subj.to_excel("F:/Project_AD/Data_excel/ALPS/PSMD_DKI_orig.xlsx", index=True, engine="openpyxl")